# RAG 

# Initialisation

In [10]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
	# Commandes à exécuter uniquement sur Google Colab
	!git clone https://github.com/Zbrlaa/Rag
	%cd Rag
	!pip install -r requirements.txt
else:
	print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Cloning into 'Rag'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 124 (delta 13), reused 45 (delta 11), pack-reused 75 (from 1)
Receiving objects: 100% (124/124), 94.62 MiB | 48.33 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/Rag/Rag/Rag


# Exercice 1 : Indexation

In [16]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("data/", use_multithreading=True)
documents = loader.load()
print(f"{len(documents)} documents")

100%|██████████| 65/65 [01:30<00:00, 10.58it/s]

65 documents


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
	chunk_size=1000,
	chunk_overlap=200,
	add_start_index=True,
	separators=["\n\n", "\n", " ", ""] #Ordre de priorité des caractères de découpe
)

all_splits = text_splitter.split_documents(documents)

print(f"Nombre de paragraphes : {len(all_splits)}")

Nombre de paragraphes : 8919


In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
	model_name="intfloat/multilingual-e5-base"
)

/tmp/ipython-input-464981128.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
	documents=all_splits, 
	embedding=embedding_model,
	persist_directory="./chroma_db"
)

print(f"Nombre d'éléments dans la base : {vectorstore._collection.count()}")

100%|██████████| 65/65 [11:56<00:00, 11.02s/it]


# Exercice 2 : Interrogation

In [ ]:
def ask_base(query):
    # Recherche docs les plus proches et scores de distance
    docs_and_scores = vectorstore.similarity_search_with_score(query)
    
    return docs_and_scores